<a href="https://colab.research.google.com/github/namwootree/Portfolio/blob/main/Alphaco_(Deep_Learning_Boot_Camp)/Long-Term%20Program/Text_Summarization/%5BMain%5D_Modeling_%26_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sat Jun 18 18:29:41 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   61C    P0    36W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q datasets transformers rouge-score nltk sentencepiece wandb

In [ ]:
from transformers import AutoConfig, AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, EarlyStoppingCallback
from datasets import load_dataset, load_metric, Dataset
import wandb
import pandas as pd
import numpy as np
import re
import nltk
import torch
import os

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Load Model & Tokenizer & Metric

In [ ]:
model_name = 'ainize/kobart-news'
metric_name = 'rouge'

In [ ]:
config = AutoConfig.from_pretrained(model_name)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [ ]:
del config.label2id
del config.id2label
del config.task_specific_params

# config.max_position_embeddings=2050

In [ ]:
config

BartConfig {
  "_name_or_path": "ainize/kobart-news",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.1,
  "d_model": 768,
  "decoder_attention_heads": 16,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "do_blenderbot_90_layernorm": false,
  "dropout": 0.1,
  "encoder_attention_heads": 16,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 1,
  "extra_pos_embeddings": 2,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "init_std": 0.02,
  "is_encoder_decoder": true,
  "max_position_embeddings": 1026,
  "model_type": "bart",
  "normalize_before": false,
  "normalize_embedding": true,
  "n

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config=config)
tokenizer = AutoTokenizer.from_pretrained(model_name)
metric = load_metric(metric_name)

### Declear Functions

In [ ]:
prefix = ""
# prefix = "summarize: "

max_input_length = 1026
max_target_length = 514

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["context"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

### Load Data

In [ ]:
train_path = '/content/drive/MyDrive/장기 프로젝트/문서 요약/augmented_data_set_247756.csv'
eval_path = '/content/drive/MyDrive/장기 프로젝트/문서 요약/train_df.csv'

In [ ]:
train_df = pd.read_csv(train_path).iloc[:150000]

eval_df = pd.read_csv(eval_path)

In [ ]:
train_df.shape, eval_df.shape

((150000, 3), (2994, 3))

In [ ]:
split_ratio = 0.1
train_dataset = Dataset.from_pandas(train_df).shuffle(seed=100)
eval_dataset = Dataset.from_pandas(eval_df).train_test_split(split_ratio, seed=100)['test']

In [ ]:
print(train_dataset)
print(eval_dataset)

Dataset({
    features: ['Unnamed: 0', 'context', 'summary'],
    num_rows: 150000
})
Dataset({
    features: ['context', 'evidence', 'summary'],
    num_rows: 300
})


In [ ]:
train_dataset = train_dataset.map(preprocess_function, 
                                  batched=True, 
                                  num_proc=4, 
                                  remove_columns=train_dataset.column_names)

eval_dataset = eval_dataset.map(preprocess_function, 
                                batched=True, 
                                num_proc=4, 
                                remove_columns=eval_dataset.column_names)

Parameter 'function'=<function preprocess_function at 0x7f6720411170> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


#0:   0%|          | 0/38 [00:00<?, ?ba/s]

#1:   0%|          | 0/38 [00:00<?, ?ba/s]

#2:   0%|          | 0/38 [00:00<?, ?ba/s]

#3:   0%|          | 0/38 [00:00<?, ?ba/s]

In [ ]:
print(train_dataset)
print(eval_dataset)

### Logging and WandB Configs

In [ ]:
wandb.login()

In [ ]:
output_dir = "./log"

In [ ]:
%env WANDB_PROJECT=BART-Generative-Summarization
report_to="wandb"
run_name="2994-Samples-Augmented-to-150000-Samples"

### Training

In [ ]:
num_train_epochs = 3

per_device_train_batch_size = 1
per_device_eval_batch_size = 1

gradient_accumulation_steps = 1

es = EarlyStoppingCallback(early_stopping_patience=8)
save_total_limit = 10
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    seed=100,

    output_dir=output_dir,

    num_train_epochs=num_train_epochs,
    learning_rate=2e-5,

    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    
    gradient_accumulation_steps=gradient_accumulation_steps,
    # fp16=True,

    save_total_limit=save_total_limit,
    save_strategy='steps',
    evaluation_strategy="steps",
    save_steps=7500,
    eval_steps=7500,

    logging_strategy='steps',
    logging_first_step=True,
    logging_steps= 500,

    weight_decay=0.01,

    lr_scheduler_type='cosine',
    warmup_ratio=0.1,

    metric_for_best_model='eval_rouge1',
    load_best_model_at_end=True,

    predict_with_generate=True,
    generation_max_length=512,
    generation_num_beams=5,

    report_to=report_to,
    run_name=run_name,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[es],
)

In [ ]:
trainer.train()
# wandb.finish()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 150000
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 450000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
7500,0.846600,0.564626,60.635200,42.262800,60.261700,60.375900,37.066700
15000,0.565100,0.357319,63.800500,45.048200,63.435500,63.583800,36.860000
22500,0.365700,0.201218,66.329500,47.738900,66.030000,66.044400,37.706700
30000,0.197900,0.104445,68.196900,48.887000,67.766800,67.953200,38.743300
37500,0.152100,0.074487,69.957700,50.303600,69.798700,69.760200,38.996700
45000,0.116800,0.061620,70.152100,50.686900,70.108000,69.825300,39.263300
52500,0.098000,0.050459,69.965700,49.923600,69.735100,69.961900,38.453300
60000,0.071200,0.044529,70.331900,51.909900,70.208800,70.362600,39.756700
67500,0.062600,0.028301,71.949500,53.286300,71.914900,71.851300,39.706700
75000,0.051200,0.029703,71.582100,53.267400,71.628700,71.519300,40.346700


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 1
Saving model checkpoint to ./log/checkpoint-7500
Configuration saved in ./log/checkpoint-7500/config.json
Model weights saved in ./log/checkpoint-7500/pytorch_model.bin
tokenizer config file saved in ./log/checkpoint-7500/tokenizer_config.json
Special tokens file saved in ./log/checkpoint-7500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch 

eval/gen_len,▁▁▃▅▅▆▄▇▇█▇▇█▇▇▇▇▇
eval/loss,█▅▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁
eval/rouge1,▁▃▄▅▆▆▆▇▇▇████████
eval/rouge2,▁▃▄▅▅▆▅▆▇▇▇███████
eval/rougeL,▁▃▄▅▆▆▆▆▇▇████████
eval/rougeLsum,▁▃▄▅▆▆▆▇▇▇████████
eval/runtime,▄▃▂█▄▇▁▇▄▇▆▁▃▂▂▅▂▁
eval/samples_per_second,▅▆▇▁▅▂█▂▅▂▃█▆▇▇▄▇█
eval/steps_per_second,▅▆▇▁▅▂█▂▅▂▃█▆▇▇▄▇█
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


### Resume Training from CKPT

In [ ]:
trainer.train('/content/drive/MyDrive/checkpoint-135000')
wandb.finish()

Loading model from /content/drive/MyDrive/checkpoint-135000.
The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 150000
  Num Epochs = 3
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 450000
  Continuing training from checkpoint, will skip to saved global_step
  Continuing training from epoch 0

  0%|          | 0/135000 [00:00<?, ?it/s]

Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
142500,0.023600,0.005599,73.063900,54.669900,72.894400,72.969400,39.920000
150000,0.022100,0.005130,73.111100,55.000000,73.000000,73.000000,39.890000
157500,0.013400,0.004135,73.111100,54.733300,72.888900,73.000000,39.823300
165000,0.012600,0.004049,73.083300,54.909100,72.916700,72.944400,39.703300
172500,0.014900,0.007563,73.044400,54.666700,72.822200,72.950000,39.830000
180000,0.014200,0.009425,72.924100,54.545000,72.729600,72.777800,39.893300
187500,0.008700,0.007579,72.879900,54.409500,72.743800,72.811000,40.026700


The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 1
Saving model checkpoint to ./log/checkpoint-142500
Configuration saved in ./log/checkpoint-142500/config.json
Model weights saved in ./log/checkpoint-142500/pytorch_model.bin
tokenizer config file saved in ./log/checkpoint-142500/tokenizer_config.json
Special tokens file saved in ./log/checkpoint-142500/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: token_type_ids. If token_type_ids are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 30

eval/gen_len,▆▅▄▁▄▅█
eval/loss,▃▂▁▁▆█▆
eval/rouge1,▇██▇▆▂▁
eval/rouge2,▄█▅▇▄▃▁
eval/rougeL,▅█▅▆▃▁▁
eval/rougeLsum,▇██▆▆▁▂
eval/runtime,▇█▁▅▂▇▂
eval/samples_per_second,▂▁█▄▇▂▇
eval/steps_per_second,▂▁█▄▇▂▇
train/epoch,▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███


In [ ]:
trainer.train('/content/log/checkpoint-187500')
wandb.finish()

### Save Model

In [ ]:
gdrive_path = '/content/drive/MyDrive/Data Science/알파코 딥러닝 부트캠프/프로젝트/AI 기반 회의 녹취록 요약 경진대회'
save_name = 'con2sum ainize-kobart-news 2994-Samples-Augmented-to-150000-Samples run 3 (from ckpt 187500)'

In [ ]:
model.save_pretrained(f'{gdrive_path}/save/{save_name}')
tokenizer.save_pretrained(f'{gdrive_path}/save/{save_name}')

### Load Model

In [ ]:
gdrive_path = '/content/drive/MyDrive/문서 요약'

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(f'{gdrive_path}/save')
tokenizer = AutoTokenizer.from_pretrained(f'{gdrive_path}/save')

In [ ]:
per_device_train_batch_size = 1
per_device_eval_batch_size = 1
num_train_epochs = 10
es = EarlyStoppingCallback(early_stopping_patience=3)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model, padding=True)

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="./log",

    num_train_epochs=num_train_epochs,
    learning_rate=2e-5,

    per_device_train_batch_size=per_device_train_batch_size,
    # gradient_accumulation_steps=16,

    per_device_eval_batch_size=per_device_eval_batch_size,
    evaluation_strategy="epoch",

    save_strategy='epoch',
    save_total_limit=3,

    # fp16=True,

    weight_decay=0.01,
    # lr_scheduler_type='linear',
    # warmup_ratio=0.1,

    metric_for_best_model='eval_loss',
    load_best_model_at_end=True,

    predict_with_generate=True,
    # generation_max_length=512,
    # generation_num_beams=5,
)

In [ ]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=data_collator
)

In [ ]:
test_path = '/content/drive/MyDrive/장기 프로젝트/문서 요약/test_df.csv'

test_df = pd.read_csv(test_path, index_col=False)
test_df.drop(['Unnamed: 0', 'id', 'title', 'region', 'agenda', 'total'], axis=1, inplace=True)
dataset = Dataset.from_pandas(test_df)

dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
preds = trainer.predict(dataset)

The following columns in the test set don't have a corresponding argument in `MT5ForConditionalGeneration.forward` and have been ignored: summary, context. If summary, context are not expected by `MT5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 300
  Batch size = 1


In [ ]:
summary = []

for pred in preds.predictions:
    pred = tokenizer.decode(pred, skip_special_tokens=True)
    summary.append(pred)

In [ ]:
sample_path = '/content/drive/MyDrive/장기 프로젝트/문서 요약/sample_submission.csv'

result = pd.read_csv(sample_path)
result['summary'] = summary

result.to_csv('result_kobart-news_24만개_6.csv', index=False)